In [37]:
from dotenv import load_dotenv
load_dotenv()  # This loads variables from .env

False

Verify your setup by running a simple script that prints out the OpenAI API key (without exposing it):

In [38]:
import os
print("Your API Key is set:", bool(os.getenv("OPENAI_API_KEY")))

Your API Key is set: False


## 1. Building a basic chain

In [39]:
from langchain.chains import TransformChain

# Define a simple function that reverses a string as an example.
def reverse_text(text: str) -> str:
    return text[::-1]

# Create a transform function
def transform_func(inputs):
    text = inputs["text"]
    return {"output": reverse_text(text)}

# Create a chain that uses this function
class ReverseChain(TransformChain):
    def __init__(self):
        super().__init__(
            input_variables=["text"],
            output_variables=["output"],
            transform=transform_func
        )

# Instantiate and test the chain
chain = ReverseChain()
result = chain({"text": "Hello LangChain"})
print("Reversed Text:", result["output"])

Reversed Text: niahCgnaL olleH


## 2. LLM Integration

In [40]:
from langchain_ollama.llms import OllamaLLM

# Create an Ollama LLM instance
llm = OllamaLLM(model="gemma3:4b")

# Define a simple prompt and get a completion
prompt = "Tell me a fun fact about LangChain."
response = llm.invoke(prompt)
print("LLM Response:", response)

LLM Response: Okay, here's a fun fact about LangChain:

**LangChain was initially conceived as a way to make it easier to build "conversational AI agents" – essentially, chatbots that could actually *reason* and *remember* things across multiple turns of a conversation.**

Initially, the project started as a simple library for connecting LLMs (Large Language Models) to data sources. However, the team quickly realized the *real* power lay in building agents that could chain together different LLM calls, tools, and memory to create truly interactive and intelligent conversational experiences. 

**The "Chain" in LangChain refers to this ability to link together different components to create complex workflows.**

Pretty cool, right? 

---

Would you like to know more about a specific aspect of LangChain, like its components or use cases?


## 3. Prompting

In [41]:
from langchain.prompts import PromptTemplate

# Create a prompt template with variables
template = "Translate the following English text to French: {text}"
prompt = PromptTemplate(input_variables=["text"], template=template)

# Format the prompt with your input text
formatted_prompt = prompt.format(text="How are you today?")
print("Formatted Prompt:", formatted_prompt)

Formatted Prompt: Translate the following English text to French: How are you today?


## 4. Adding memory

In [ ]:
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate
from langchain.memory import ChatMessageHistory

# Create a simple chain with the prompt template and LLM
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant."),
    ("human", "{input}"),
])
chain = prompt | llm

# Create the runnable with message history
conversation_with_memory = RunnableWithMessageHistory(
    chain,
    lambda session_id: ChatMessageHistory(),  # Function to get history for a session
    input_messages_key="input",
    history_messages_key="history"
)

# Simulate a conversation with a session ID
session_id = "example_session"

# First interaction
print("User: Hi, who are you?")
response = conversation_with_memory.invoke(
    {"input": "Hi, who are you?"}, 
    config={"configurable": {"session_id": session_id}}
)
print("Assistant:", response)

# Second interaction that references the history
print("User: Can you remind me what we talked about?")
response = conversation_with_memory.invoke(
    {"input": "Can you remind me what we talked about?"}, 
    config={"configurable": {"session_id": session_id}}
)
print("Assistant:", response)

User: Hi, who are you?
Assistant: Hi there! I'm Gemma, a large language model created by the Gemma team at Google DeepMind. I’m an open-weights model, which means I’m widely available for public use! 

It’s nice to meet you. 😊 

What can I do for you today?
User: Can you remind me what we talked about?
Assistant: Please provide me with a little more context! I have no memory of our previous conversations unless you give me a starting point. 😊 

To help me remind you, could you tell me:

*   **What were we talking about?** (e.g., "We were discussing travel plans," or "We were brainstorming ideas for a story.")
*   **When did we last talk?** (e.g., "Just a few minutes ago," or "Yesterday afternoon.")

The more information you give me, the better I can help you!


## 5. Agents & Tools

note: this implementation is not the most modern one, but it is the most simple one. 

In [43]:
from langchain.agents import initialize_agent, Tool

# Define a simple calculator tool
def calculator_tool(query: str) -> str:
    try:
        result = eval(query)  # Caution: eval() should only be used with trusted input.
        return str(result)
    except Exception as e:
        return f"Error: {str(e)}"

calculator = Tool(
    name="Calculator",
    func=calculator_tool,
    description="Performs basic arithmetic calculations."
)

# Initialize an agent with the tool
agent = initialize_agent([calculator], llm, agent="zero-shot-react-description", verbose=True)

# Test the agent with a math query
query = "What is 15 * 3?"
result = agent.run(query)
print("Agent Response:", result)



> Entering new AgentExecutor chain...
I need to calculate 15 multiplied by 3.
Action: Calculator
Action Input: 15 * 3
Observation: 45
Thought:I now know the final answer
Final Answer: 45

> Finished chain.
Agent Response: 45


## 6. RAG

In [44]:
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain_community.embeddings import OllamaEmbeddings  # or another embedding model
from langchain_ollama.llms import OllamaLLM  # assuming you're using Ollama

# Create an embedding model
embeddings = OllamaEmbeddings(model="all-minilm")  # or another suitable model
llm = OllamaLLM(model="gemma3:4b")  # or another suitable model

# Create a custom loader class to handle potential errors
class ErrorHandlingTextLoader(TextLoader):
    def load(self):
        try:
            return super().load()
        except Exception as e:
            print(f"Error loading {self.file_path}: {str(e)}")
            return []

# Load all text files from a directory
loader = DirectoryLoader(
    ".././docs/", 
    glob="**/*.txt",  # This pattern matches all .txt files, including in subdirectories
    loader_cls=ErrorHandlingTextLoader,
    show_progress=True
)
# Create the index with the embedding model
index = VectorstoreIndexCreator(
    embedding=embeddings,
    vectorstore_kwargs={"collection_name": "langchain_docs"},
).from_loaders([loader])


# Query the index and get a response
query = "Make a haiku about who discovered the Uncertainty Principle"
docs = index.query(query, llm=llm)  # Note: Using query method instead of search
print("Retrieved Documents:", docs)

# If you want to manually process the docs and send to LLM
docs = index.vectorstore.similarity_search(query)
combined_text = " ".join([doc.page_content for doc in docs])
final_prompt = f"Based on the following context, answer the query: {combined_text}\n\nQuery: {query}"
response = llm.invoke(final_prompt)  # Using invoke() instead of direct call
print("Final Answer:", response)

/Users/nojeda/repo/LangchainLearning/.venv/lib/python3.13/site-packages/langchain/indexes/vectorstore.py:171: UserWarning: Using InMemoryVectorStore as the default vectorstore.This memory store won't persist data. You should explicitlyspecify a vectorstore when using VectorstoreIndexCreator
  warnings.warn(
100%|██████████| 2/2 [00:00<00:00, 2083.61it/s]


Retrieved Documents: Heisenberg’s insight,
Limits what we know of all,
Quantum world unfolds.
Final Answer: Here's a haiku based on the provided text about the Uncertainty Principle:

Heisenberg’s insight,
Limits what we know of space,
Time and place entwined.
